## 2 Toxic Words

In [1]:
# Load required packages
import nltk
import math
import decimal
import pandas as pd
from pandas import read_csv
from os import listdir
from os.path import isfile, join
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import *

### 2.2 Data 

In [41]:
whole_data = pd.read_csv("E:/GW/Textbook/Natural Language Processing/project/jigsaw-unintended-bias-in-toxicity-classification/train.csv")
train_set = whole_data[0:500000] # Train set

In [54]:
# Obtain the validation set
corpus = whole_data[1000000:1006230] 
validation_set = []
numb1 = 0
numb0 = 0
for row in corpus.itertuples():
    if getattr(row, 'target') >=0.7:
        numb1 +=1 
        #print(getattr(row, 'target'))
        validation_set += [(getattr(row, 'comment_text'), math.ceil(getattr(row, 'target')))]
for row in corpus.itertuples():    
    if   numb0 <numb1:
        if getattr(row, 'target') ==0:
            #print(getattr(row, 'target'))
            validation_set += [(getattr(row, 'comment_text'), math.floor(getattr(row, 'target')))]
            numb0 +=1
        
print(len(validation_set))
print(numb1)
print(numb0)

500
250
250


In [70]:
# Obtain the test set
corpus = whole_data[1500000:1506840] 
test_set = []
numb1 = 0
numb0 = 0
for row in corpus.itertuples():
    if getattr(row, 'target') >=0.7:
        numb1 +=1 
        #print(getattr(row, 'target'))
        test_set += [(getattr(row, 'comment_text'), math.ceil(getattr(row, 'target')))]
for row in corpus.itertuples():    
    if   numb0 <numb1:
        if getattr(row, 'target') ==0:
            #print(getattr(row, 'target'))
            test_set += [(getattr(row, 'comment_text'), math.floor(getattr(row, 'target')))]
            numb0 +=1
        
print(len(test_set))
print(numb1)
print(numb0)

500
250
250


### 2.3 Application
#### 2.3.1 The first approach

In [ ]:
corpus = train_set[0:50000]# Get 10 sub-groups of train set
# like [0:50000], [50001:100000],[100001:150000].... [450001:500000]
group = []
numb1 = 0
numb0 = 0
for row in corpus.itertuples():
    if getattr(row, 'target') >=0.7:
        numb1 +=1 
        #print(getattr(row, 'target'))
        group += [(getattr(row, 'comment_text'), math.ceil(getattr(row, 'target')))]
for row in corpus.itertuples():    
    if   numb0 <numb1:
        if getattr(row, 'target') ==0:
            #print(getattr(row, 'target'))
            group += [(getattr(row, 'comment_text'), math.floor(getattr(row, 'target')))]
            numb0 +=1
        
print(len(group))
print(numb1)
print(numb0)

In [ ]:
#####  step 1 Take the difference between tokens of sub-train set(group) and clean corpus to get potential toxic tokens.
# The function to detect the noise tokens
min_token_length = 2
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def isNoise(token):     
    is_noise = False
    if porter_stemmer.stem(token.lower())in stopWords:
        is_noise = True
    elif len(token) < min_token_length:
        is_noise = True
    elif re.findall(r"\`|\'|/", token):
        is_noise = True
    elif re.findall(r"[\d]",token):
        is_noise = True  
    elif re.findall(r"^\.",token):
        is_noise = True  
    elif re.findall(r"^\.",token):
        is_noise = True  
    elif not re.findall(r"[A-Za-z0-9]",token):
        is_noise = True
    elif re.findall(r"\…",token):
        is_noise = True
    return is_noise 

# Tokenization preparation
treebank_tokenizer = TreebankWordTokenizer()
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stopWords = set(stopwords.words('english'))
porter_stemmer = PorterStemmer()

# The functio to load Reuters data in NLTK
def readdoc(data):
    doc_texts = ""
    for doc in data: 
        doc_texts += reuters.open(doc).read()
    return doc_texts

clean_corpus = readdoc(reuters.fileids()) # Load reuters data
clean_tokens = treebank_tokenizer.tokenize(clean_corpus) # Word tokenization
clean_corpus_tokens_d  = [porter_stemmer.stem(word.lower()) for word in clean_tokens]# Lematization and filter out noise tokens
clean_corpus_tokens = set(clean_corpus_tokens_d) # Delete the duplicated tokens
print(len(clean_corpus_tokens)) # Show the length of the clean corpus 

# Tokens of each of the 10 groups of train_set
tokens=set(word.lower() for words in group for word in word_tokenize(words[0]) if not isNoise(word))
print(len(tokens))

# potential toxic tokens
potential_toxic_tokens= [word for word in tokens if porter_stemmer.stem(word.strip(r"^[\-]+\.")) not in clean_corpus_tokens]
print(len(set(potential_toxic_tokens)))

In [ ]:
#####  step 2 Apply Naïve Bayes and get the informative tokens of each  of  these 10  groups

In [ ]:
# Apply the Naive Bayes
train_w = [({word: (word in word_tokenize(x[0])) for word in set(potential_toxic_tokens)}, x[1]) for x in group] 
model = nltk.NaiveBayesClassifier.train(train_w)

# The function to output the informative tokens
def show_most_informative_features_in_listtest(classifier, n=10):
    """
    Return a nested list of the "most informative" features 
    used by the classifier along with it's predominant labels
    """
    cpdist = classifier._feature_probdist       # probability distribution for feature values given labels
    feature_list = []
    for (fname, fval) in classifier.most_informative_features(n):
        def labelprob(l):
            return cpdist[l, fname].prob(fval)
        labels = sorted([l for l in classifier._labels if fval in cpdist[l, fname].samples()], 
                        key=labelprob)
        if (cpdist[labels[-1], fname].prob(fval) != cpdist[labels[0], fname].prob(fval)) :
            if labels[-1] ==1 & fval == True:
                feature_list.append(fname)
            if  fval == False:
                if labels[-1] ==0:
                    feature_list.append(fname)
    return feature_list


possible_toxic_words = show_most_informative_features_in_listtest(model,1000)
print(possible_toxic_words)


t5wset = set(possible_toxic_words) # use this code to delete the duplicated words in the each group's possible toxic words
# Then we can get t5wset, t0510set, t1015set, ..., t4550set. Ten groups of possible_toxic_words. 

# Since it will take about 100 minutes to get each geoup of possible_toxic_words, it is hard to get all 10 groups at
# one run. Then we need to output each group of possible_toxic_words in excel form for convenince.
t5w = pd.DataFrame.from_dict({
    'Toxic_word': list(t5wset)
})
t5w.to_csv('E:/GW/Textbook/Natural Language Processing/project/t5w.csv', index=False)
# Then we can get t5w, t0510, t1015, ..., t4550. Ten groups of possible_toxic_words with excel form.

In [ ]:
#####  step 3 Find the best boundary based on the validation set

In [55]:
# Process the validation set.
validation_tokens=[word.lower() for words in validation_set for word in word_tokenize(words[0]) if not isNoise(word)]
print(len(validation_tokens))
real_validation = []
for i in validation_tokens:
    real_validation += [i.strip('\.')]
print(len(real_validation))

# Load the annotated toxic words
validation_reference = pd.read_csv("E:/GW/Textbook/Natural Language Processing/project/validation_reference.csv")

12524
12524


In [72]:
# Read 10 groups of possible_toxic_words
dir_base = "E:/GW/Textbook/Natural Language Processing/project/bayes/"

def read_file(filename):
    input_file_text = read_csv(filename)
    return input_file_text
   
def read_directory_files(directory):
    file_texts = []
    allwords = []
    files = [f for f in listdir(directory) if isfile(join(directory, f))]
    for f in files:
        file_texts = read_file(join(directory, f))
        for row in file_texts.itertuples():
            allwords += [(getattr(row, 'Toxic_word'))] 
        #file_texts += list(file_text)
    return allwords
    
ten_groups_possible_toxic_words = read_directory_files(dir_base) # Get 10 groups of posiible toxic words

# Calculate the frequency of each words in ten_groups_possible_toxic_words
freq_ten_groups_possible_toxic_words = nltk.FreqDist(ten_groups_possible_toxic_words)

# Show the most_common 20 words
freq_ten_groups_possible_toxic_words.most_common(20)

[('unfit', 10),
 ('idiot', 10),
 ('moronic', 10),
 ('bigoted', 10),
 ('scumbags', 10),
 ('dumb', 10),
 ('useless', 10),
 ('coward', 10),
 ('racists', 10),
 ('morons', 10),
 ('woman', 10),
 ('uneducated', 10),
 ('liars', 10),
 ('traitor', 10),
 ('disgusting', 10),
 ('garbage', 10),
 ('shit', 10),
 ('hypocritical', 10),
 ('hypocrisy', 10),
 ('despicable', 10)]

In [ ]:
# The function to calculate precision and recall
def precison_recall(real_toxic,real_evaluation,evaluation_reference):
    pre_toxic = []
    for i in real_evaluation:
        if i in real_toxic:
            pre_toxic += [i]
    reference = []
    for row in evaluation_reference.itertuples():
            reference += [(getattr(row, 'TOX')).lower()]
    consist = []
    for i in reference:
        if i in pre_toxic:
            consist += [i]
    contrast = []
    for i in pre_toxic:
        if i not in reference:
            contrast += [i]
    # Precision and recall
    Precision = len(consist)/(len(consist)+len(contrast))
    Recall = len(consist)/(len(reference))
    print(Precision,Recall)
    return

In [82]:
# Calculate the precision and recall
possible = []
for i in set(ten_groups_possible_toxic_words):
    if freq_ten_groups_possible_toxic_words[i] >9:# Here we can change the number to compare different boundaries' performance. 
        possible += [i]
real_toxic = []
for i in possible:
    real_toxic += [(str(i)).strip('\.')]
precison_recall(real_toxic,real_validation,validation_reference)

0.9070796460176991 0.5061728395061729


#### 2.3.2 The second approach

In [ ]:
#### Step 1: Split the train_set into clean_group and toxic_group

In [43]:
len(train_set)

500000

In [57]:
clean_group = []
toxic_group = []
numb1 = 0
numb0 = 0
for row in train_set.itertuples():
    if getattr(row, 'target') >=0.7:
        numb1 +=1 
        toxic_group += [(getattr(row, 'comment_text'), math.ceil(getattr(row, 'target')))]
for row in train_set.itertuples(): 
    if getattr(row, 'target') ==0:
        clean_group += [(getattr(row, 'comment_text'), math.floor(getattr(row, 'target')))]
        numb0 +=1
        
print(len(clean_group),len(toxic_group))

352824 16276


In [ ]:
#### Step 2: Tokenize clean_group and T_group

In [58]:

clean_tokens=[word.lower() for words in clean_group for word in word_tokenize(words[0]) if not isNoise(word)]
len(clean_tokens)

9572373

In [59]:
T_tokens=[word.lower() for words in toxic_group for word in word_tokenize(words[0]) if not isNoise(word)]
len(T_tokens)

365845

In [ ]:
#### Step 3: Calculate the ratio

In [60]:
freq_clean_tokens = nltk.FreqDist(clean_tokens)
freq_T_tokens = nltk.FreqDist(T_tokens)
freq_T_tokens.most_common(20)

[('this', 4950),
 ('people', 3071),
 ('trump', 3015),
 ('was', 2971),
 ('stupid', 2741),
 ('like', 2695),
 ('his', 2577),
 ('has', 2107),
 ('would', 1966),
 ('one', 1770),
 ('get', 1766),
 ('because', 1384),
 ('think', 1240),
 ('only', 1204),
 ('know', 1200),
 ('does', 1171),
 ('idiot', 1156),
 ('us', 1153),
 ('time', 1074),
 ('any', 1063)]

In [66]:
toxic = []
uniq = []
for i in set(T_tokens):
    if freq_clean_tokens[i] ==0:
        uniq += [i]
        if freq_T_tokens[i] >3:
            if (((freq_T_tokens[i]/len(T_tokens))*((numb1)/(numb1+numb0)))/(((freq_clean_tokens[i]+0.1)/len(clean_tokens))*((numb0)/(numb1+numb0))))>1.3:
                toxic += [i]
    elif freq_clean_tokens[i] !=0:
        if (((freq_T_tokens[i]/len(T_tokens))*((numb1)/(numb1+numb0)))/(freq_clean_tokens[i]/len(clean_tokens))*((numb0)/(numb1+numb0)))>1.3:# The threshold here can be adjusted to find the best one
            toxic += [i]

In [85]:
# Calculate the precision and recall
real_toxic = []
for i in toxic:
    real_toxic += [(str(i)).strip('\.')]
precison_recall(real_toxic,real_validation,validation_reference)

0.955719557195572 0.6395061728395062


### 2.4 Evaluation

In [76]:
# Load the test annotated toxic words
test_reference = pd.read_csv("E:/GW/Textbook/Natural Language Processing/project/test_reference.csv")


In [80]:
# Process the test_set
test_tokens=[word.lower() for words in test_set for word in word_tokenize(words[0]) if not isNoise(word)]
print(len(test_tokens))
real_test = []
for i in test_tokens:
    real_test += [i.strip('\.')]

11385


In [81]:
# First approach
possible = []
for i in set(ten_groups_possible_toxic_words):
    if freq_ten_groups_possible_toxic_words[i] >9:# Here we can change the number to compare different boundaries' performance. 
        possible += [i]
first_real_toxic = []
for i in possible:
    first_real_toxic += [(str(i)).strip('\.')]
precison_recall(first_real_toxic,real_test,test_reference)

0.9094827586206896 0.5877437325905293


In [86]:
# Second approach
second_real_toxic = []
for i in toxic:
    second_real_toxic += [(str(i)).strip('\.')]
precison_recall(second_real_toxic,real_test,test_reference)

0.9148936170212766 0.5988857938718662


### 2.5 All toxic words of whole dataset

In [88]:
W_toxic_group = []
W_clean_group = []
numb1 = 0
numb0 = 0
for row in whole_data.itertuples():
    if getattr(row, 'target') >=0.7:
        numb1 +=1 
        W_toxic_group += [(getattr(row, 'comment_text'), math.ceil(getattr(row, 'target')))]
for row in whole_data.itertuples(): 
    if getattr(row, 'target') ==0:
        W_clean_group += [(getattr(row, 'comment_text'), math.floor(getattr(row, 'target')))]
        numb0 +=1
print(numb1)
print(numb0)

58674
1264764


In [89]:
W_clean_tokens=[word.lower() for words in W_clean_group for word in word_tokenize(words[0]) if not isNoise(word)]
print(len(W_clean_tokens))
W_T_tokens=[word.lower() for words in W_toxic_group for word in word_tokenize(words[0]) if not isNoise(word)]
print(len(W_T_tokens))

freq_W_clean_tokens = nltk.FreqDist(W_clean_tokens)
freq_W_T_tokens = nltk.FreqDist(W_T_tokens)

W_toxic = []
W_uniq = []
for i in set(W_T_tokens):
    if freq_W_clean_tokens[i] ==0:
        W_uniq += [i]
        if freq_W_T_tokens[i] >3:
            if (((freq_W_T_tokens[i]/len(W_T_tokens))*((numb1)/(numb1+numb0)))/(((freq_W_clean_tokens[i]+0.1)/len(W_clean_tokens))*((numb0)/(numb1+numb0))))>1.3:
                W_toxic += [i]
    elif freq_W_clean_tokens[i] !=0:
        if (((freq_W_T_tokens[i]/len(W_T_tokens))*((numb1)/(numb1+numb0)))/(freq_W_clean_tokens[i]/len(W_clean_tokens))*((numb0)/(numb1+numb0)))>1.3:# The threshold here can be adjusted to find the best one
            W_toxic += [i]
print(len(W_toxic))
W_toxic

33469379
1297829
673


['dullards',
 'arses',
 'fucker',
 'scums',
 'agian',
 '*everywhere*',
 'scumbags',
 'laguillotine',
 'batsh*t',
 'lilly-white',
 'horseshit',
 'wavetaker',
 'loudmouthed',
 'p***y',
 'hahahahahahahah',
 'prohibtion',
 'pussies',
 'oafish',
 'cow-pie',
 'alah',
 '*please*',
 'prohibes',
 'sh*t.',
 'jinxed',
 'slags',
 'builts',
 'stoopid',
 'clitoris',
 'chickenshit',
 'con-artist',
 'slut',
 'enviro-zealots',
 'waaah',
 'non-comment',
 'tiny-minded',
 'boy-child',
 'tard',
 'self-loving',
 '-h.l',
 'cking',
 'lannatullahi',
 'cuck',
 'pussy-grabbing',
 'jerkoff',
 'damnable',
 'magaphant',
 'dipshit',
 'shimmy',
 'imbecilic',
 'dork',
 'presudent',
 'trumpaloons',
 'ribald',
 'troll',
 'libatards',
 'ill-spirited',
 'trudaeu',
 'fing',
 'inth',
 'beautiful.',
 'drug-alcohol',
 'whorehouse',
 'stupid..',
 'half-witted',
 'f***',
 'non-readers',
 'muslum',
 'supremicists',
 'dums',
 'nasionaal',
 'lhao',
 'dickhead',
 'lgbtx',
 'troll..',
 'lesbo',
 'krooked',
 'scrotum',
 'kowalke',
 '